In [1]:
import pandas as pd
import librosa
import soundfile as sf
import os

In [2]:
PATH = "../Data/" # "../../../desarrollo/Data/"
OUTPUT_FOLDER = PATH + "Dataset/Audios/BirdNet/train/"

train_csv = PATH + "Dataset/CSVs/train_with_bg.csv"

In [3]:
# Read the csv
df = pd.read_csv(train_csv)

In [5]:
# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    audio_path = row['path']
    audio_path = PATH + "Dataset/Audios/" + audio_path
    start_time = row['start_time']
    end_time = row['end_time']
    specie = row['specie']
    
    # Crea la carpeta para la especie si no existe
    specie_folder = os.path.join(OUTPUT_FOLDER, specie)
    if not os.path.exists(specie_folder):
        os.makedirs(specie_folder)
    
    # Carga el audio
    audio, sr = librosa.load(audio_path, sr=None)
    
    # Calcula los índices de inicio y fin en muestras
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    
    # Recorta el audio
    audio_cropped = audio[start_sample:end_sample]
    
    # Define el nombre del archivo de salida
    output_filename = os.path.join(specie_folder, os.path.basename(audio_path))
    
    # Guarda el audio recortado
    sf.write(output_filename, audio_cropped, sr)

print("Proceso completado.")

Proceso completado.


In [7]:
# Background:
# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    audio_path = row['path']
    audio_path = PATH + "Dataset/Audios/" + audio_path
    start_time = row['start_time']
    end_time = row['end_time']
    specie = row['specie']

    num_bg = 0

    if specie == "No audio":
    
        # Crea la carpeta para la especie si no existe
        background_folder_name = "Background"
        specie_folder = os.path.join(OUTPUT_FOLDER, background_folder_name)

        if not os.path.exists(specie_folder):
            os.makedirs(specie_folder)

        print(audio_path)
        print(specie_folder)
        
        # Define el nombre del archivo de salida
        output_filename = os.path.join(specie_folder, os.path.basename(audio_path))
        print(output_filename)

        # Copiar el audio en la carpeta de salida
        os.system(f"cp {audio_path} {output_filename}")

        print("\n")

        num_bg += 1

print("Proceso completado.")
print(f"Se han guardado {num_bg} audios de background.")

../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_094000 .WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230509_094000 .WAV


../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_095000 .WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230509_095000 .WAV


../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_100000 .WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230509_100000 .WAV


../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_101000 .WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230509_101000 .WAV


../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_102000 .WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230509_102000 .WAV


../Data/Dataset/Audios/AM2/2023_05_09/AM02_20230509_103000 .

cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory
cp: target '.WAV' is not a directory




../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_072000.WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230523_072000.WAV


../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_073000.WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230523_073000.WAV


../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_074000.WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230523_074000.WAV


../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_075000.WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230523_075000.WAV


../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_080000.WAV
../Data/Dataset/Audios/BirdNet/train/Background
../Data/Dataset/Audios/BirdNet/train/Background/AM02_20230523_080000.WAV


../Data/Dataset/Audios/AM2/2023_05_23/AM02_20230523_081000.WAV
../Da

In [6]:
Species_names_file = "../BirdNET_GLOBAL_6K_V2.4_Labels.txt"
species = pd.read_csv(Species_names_file, sep="\t", header=None)

In [19]:
# Change names of folders

SPECIES_FOLDER = PATH + "Dataset/Audios/BirdNet/train/"

# Change name for each subfolder
for folder in os.listdir(SPECIES_FOLDER):
    if os.path.isdir(os.path.join(SPECIES_FOLDER, folder)):
        # Lowercase
        # folder = folder.lower()

        # Ensure folder name is lowercase, first letter uppercase and if there are two words, the second word begins with lowercase
        folder = folder.lower().capitalize()
        if " " in folder:
            folder = folder.split(" ")[0] + " " + folder.split(" ")[1].lower()
        
        # Look for the row of the species that contains the folder name
        row = species[species[0].str.contains(folder)]

        # Don't change folder name if it is Bird, Curruca, Alaudidae
        No_change = ["Bird", "Curruca", "Alaudidae", "Fringilla", "Lanius", "Passer", "Sturnus", "Sylvia", "Background"]
        if folder in No_change:
            print(f"Folder {folder} not renamed because it is a general category. +++++++++++++++")
        elif folder == "No audio":
            # Rename to Background
            os.rename(os.path.join(SPECIES_FOLDER, folder), os.path.join(SPECIES_FOLDER, "Background"))
        else:
            # If the species is found, change the name of the folder
            if not row.empty:
                species_name = row.iloc[0][0]
                os.rename(os.path.join(SPECIES_FOLDER, folder), os.path.join(SPECIES_FOLDER, species_name))
                print(f"Folder {folder} renamed to {species_name}")
            else:
                print(f"Species {folder} not found in the species list. ----------------")

Folder Alaudidae not renamed because it is a general category. +++++++++++++++
Folder Anthus pratensis renamed to Anthus pratensis_Meadow Pipit
Species Background not found in the species list. ----------------
Folder Bird not renamed because it is a general category. +++++++++++++++
Folder Calandrella brachydactyla renamed to Calandrella brachydactyla_Greater Short-toed Lark
Folder Carduelis carduelis renamed to Carduelis carduelis_European Goldfinch
Folder Cettia cetti renamed to Cettia cetti_Cetti's Warbler
Folder Chloris chloris renamed to Chloris chloris_European Greenfinch
Folder Ciconia ciconia renamed to Ciconia ciconia_White Stork
Folder Cisticola juncidis renamed to Cisticola juncidis_Zitting Cisticola
Folder Curruca not renamed because it is a general category. +++++++++++++++
Folder Curruca melanocephala renamed to Curruca melanocephala_Sardinian Warbler
Folder Curruca undata renamed to Curruca undata_Dartford Warbler
Folder Cyanopica cooki renamed to Cyanopica cooki_Iberia